# VacationPy
----


In [1]:
!pip install gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
df = pd.read_csv("../output_data/weather_output.csv")
weather_df = df.drop(columns=["Unnamed: 0"])
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,mataura,5,NZ,1584494960,66,-46.19,168.86,15.00,4.02
1,ushuaia,40,AR,1584494865,47,-54.80,-68.30,13.00,4.10
2,marawi,95,PH,1584494961,75,8.00,124.29,27.78,1.79
3,albany,1,US,1584494952,80,42.60,-73.97,7.00,4.10
4,bluff,22,NZ,1584494961,66,-46.60,168.33,15.00,4.02


In [5]:
# Transform from celsius to fahrenheit
Max_Temp_F = [(x*1.8) + 32 for x in weather_df["Max Temp"]]


In [6]:
# Add to DataFrame as a new column
weather_df["Max Temp (F)"] = Max_Temp_F
weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Max Temp (F)
0,mataura,5,NZ,1584494960,66,-46.19,168.86,15.00,4.02,59.000
1,ushuaia,40,AR,1584494865,47,-54.80,-68.30,13.00,4.10,55.400
2,marawi,95,PH,1584494961,75,8.00,124.29,27.78,1.79,82.004
3,albany,1,US,1584494952,80,42.60,-73.97,7.00,4.10,44.600
4,bluff,22,NZ,1584494961,66,-46.60,168.33,15.00,4.02,59.000
...,...,...,...,...,...,...,...,...,...,...
562,payakumbuh,82,ID,1584495115,67,-0.22,100.63,25.75,1.36,78.350
563,dongsheng,100,CN,1584495115,25,39.82,109.98,7.81,11.49,46.058
564,kishtwar,0,IN,1584495115,58,33.32,75.77,-3.60,1.22,25.520
565,pizarro,14,CO,1584495115,81,4.95,-77.37,26.61,1.88,79.898


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# Humidity Heatmap

weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)
locations = weather_df[["Lat", "Lng"]].astype(float)

# Add heatmap figure to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
sum_weather_df = weather_df

# A max temperature lower than 80 degrees but higher than 70
sum_weather_df = sum_weather_df.loc[(sum_weather_df["Max Temp (F)"] < 80) & (sum_weather_df["Max Temp (F)"] > 70)]

# Wind speed less than 10 mph
sum_weather_df = sum_weather_df.loc[sum_weather_df["Wind Speed"] < 10]

# Zero cloudiness
sum_weather_df = sum_weather_df.loc[sum_weather_df["Cloudiness"] == 0]

sum_weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Max Temp (F)
31,kovur,0,IN,1584494969,91,14.48,79.98,22.32,2.85,72.176
107,dwarka,0,IN,1584494987,65,22.24,68.97,25.40,4.16,77.720
120,flinders,0,AU,1584494990,44,-34.58,150.86,25.00,3.60,77.000
122,puerto castilla,0,HN,1584494991,77,16.02,-85.97,26.41,7.63,79.538
167,batemans bay,0,AU,1584495005,65,-35.72,150.18,23.33,1.34,73.994


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
# Hotel Map

hotel_df = sum_weather_df

# Set parameters to search hotels with 5000 meters
params = {
    "key": g_key,
    "radius": 5000,
    "keyword": "hotel",
    }

for index, row in hotel_df.iterrows():
    # Get lat, lng from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Create url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(url, params=params).json()
    
    # Extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError  ):
        print("Result not found... Skipping.")
        
    print("------------")


Retrieving Results for Index 31: kovur.
Closest hotel is Minerva Grand Nellore.
------------
Retrieving Results for Index 107: dwarka.
Closest hotel is VITS Devbhumi, Dwarka.
------------
Retrieving Results for Index 120: flinders.
Closest hotel is Lakeview Hotel Motel.
------------
Retrieving Results for Index 122: puerto castilla.
Result not found... Skipping.
------------
Retrieving Results for Index 167: batemans bay.
Closest hotel is Bay Breeze Motel.
------------
Retrieving Results for Index 174: sambava.
Closest hotel is Carrefour.
------------
Retrieving Results for Index 198: cayenne.
Closest hotel is Royal Amazonia.
------------
Retrieving Results for Index 285: kiama.
Closest hotel is Bellevue Accommodation.
------------
Retrieving Results for Index 286: san vicente.
Closest hotel is Hotel Y Restaurante Los Lapachos.
------------
Retrieving Results for Index 305: dosso.
Closest hotel is Galaxy Dosso.
------------
Retrieving Results for Index 307: bubaque.
Closest hotel is Po

In [18]:
# Store hotel result into a DataFrame
hotel_df.to_csv("../output_data/hotel_output.csv")
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Max Temp (F),Hotel Name
31,kovur,0,IN,1584494969,91,14.48,79.98,22.32,2.85,72.176,Minerva Grand Nellore
107,dwarka,0,IN,1584494987,65,22.24,68.97,25.40,4.16,77.720,"VITS Devbhumi, Dwarka"
120,flinders,0,AU,1584494990,44,-34.58,150.86,25.00,3.60,77.000,Lakeview Hotel Motel
122,puerto castilla,0,HN,1584494991,77,16.02,-85.97,26.41,7.63,79.538,NaN
167,batemans bay,0,AU,1584495005,65,-35.72,150.18,23.33,1.34,73.994,Bay Breeze Motel


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in sum_weather_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map

# Add marker layer of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
symbol_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info)
fig.add_layer(symbol_layer)
# Display Map
fig
# Display Map

Figure(layout=FigureLayout(height='420px'))